# Python Client

#### Import ModelBazaar class.

In [ ]:
import thirdai
from thirdai.neural_db import ModelBazaar

# Activate your license
thirdai.licensing.activate("YOUR-THIRDAI-ACTIVATION-KEY")

### Initialise a ModelBazaar object with url where model_bazaar is hosted.

In [ ]:
bazaar = ModelBazaar(base_url="http://<headnode-ip>/api/")

### Sign up

In [ ]:
# Sends a email verification link.
bazaar.sign_up(email="your-mail-id@domain.com", password="password", username="username")

### Login

In [ ]:
bazaar.log_in(email="your-mail-id@domain.com", password="password")

### Creates a model from scratch on ModelBazaar -- trained on given list of documents.

In [ ]:
# Can be synchronous or asynchronous(default; in which case we call await_train)
# `doc_type` can be "local"(default), "nfs" or "s3".
model = bazaar.train(
    model_name="model-1",
    docs=["/path/to/fileA", "/path/to/fileB"],
    doc_type="local",
    is_async=True,
)

### Blocking call to wait till model finishes training.

In [ ]:
bazaar.await_train(model)

### Deploys the model and returns an ndb_client which can be used to interact with deployed neural_db model.

In [ ]:
# Can be synchronous or asynchronous(default; in which case we call await_deploy)
ndb_client = bazaar.deploy(
    model_identifier=model.model_identifier,
    deployment_name="deployment-1",
    is_async=True,
)

### Blocking call to wait till deployment finishes.

In [ ]:
bazaar.await_deploy(ndb_client)

### Insert new files to index in the deployed neural_db model.

In [ ]:
ndb_client.insert(
    files=["/path/to/fileC", "/path/to/fileD"]
)

### Search the ndb model.

In [ ]:
results = ndb_client.search(query="who are the authors", top_k="5")

query_id = results["query_id"]
query_text = results["query_text"]
references = results["references"]
for reference in references:
    print(reference["text"])

In [ ]:
# [RLHF] Associate queries to teach your model what are similar queries.
ndb_client.associate(
    text_pairs=[
                    {"source": "authors", "target": "contributors"},
                    {"source": "paper", "target": "document"},
                ]
)

### Associate the text pairs

In [ ]:
best_answer = references[4]
good_answer = references[2]
ndb_client.upvote(
    [
        {"query_text": query_text, "reference_id": best_answer["id"]},
        {"query_text": query_text, "reference_id": good_answer["id"]},
    ]
)

### Undeploys the model.

In [ ]:
bazaar.undeploy(ndb_client)

### Deletes the model from model bazaar.

In [ ]:
# Use `list_models()` to find models on ModelBazaar and their identifiers.
# example model_identifier : "model-author/model-name"
bazaar.delete(model_identifier=model.model_identifier)

### Return a list of models accessible to user on model bazaar.

In [ ]:
# Each model contains an attribute `model_identifier` to uniquely identify the model.
bazaar.list_models()

### Returns a list of all active deployments.

In [ ]:
# Each deployment contains an attribute `deployment_identifier` to uniquely identify the deployment.
bazaar.list_deployments()

### Connects to an active deployment.

In [ ]:
# Use `list_deployments()` to find active deployments and their identifiers.
# example deployment_identifier : "model-author/model-name:deployment-author/deployment-name"
ndb_client = bazaar.connect(deployment_identifier="your-deployment-identifier")

### Push a local NDB model to model bazaar.

In [ ]:
bazaar.push_model(
    model_name="test-upload",
    local_path="path/local/ndb/model.ndb",
    access_level="private",
)

### Pull model from model bazaar to local.

In [ ]:
# Remember, this is your local ndb_model, not the remote deployed one.
ndb_model = bazaar.pull_model(model_identifier="your-model-identifer")